In [1]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split


transform = transforms.Compose([

    transforms.ToTensor(),
    transforms.Normalize(mean = [0.5], std = [0.5])
])

data = torchvision.datasets.EuroSAT(root = './data', download = True, transform = transform)
print(data)

Dataset EuroSAT
    Number of datapoints: 20827
    Root location: ./data
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=[0.5], std=[0.5])
           )


In [2]:
classes = data.classes
print(classes)

images = []
labels = []
for img, label in data:
    images.append(img)
    labels.append(label)

device = "cuda" if torch.cuda.is_available() else "cpu"

X = torch.stack(images).to(device)
y = torch.tensor(labels).to(device)

X.
print(X.shape)
print(y.shape)

['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'Residential', 'River']
torch.Size([20827, 3, 64, 64])
torch.Size([20827])


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)
print(X.shape)

torch.Size([20827, 3, 64, 64])


In [4]:
batchSize = 32
trainDataset = torch.utils.data.TensorDataset(X_train, y_train)
trainLoader = torch.utils.data.DataLoader(trainDataset, batch_size = batchSize, shuffle = True)

testDataset = torch.utils.data.TensorDataset(X_test, y_test)
testLoader = torch.utils.data.DataLoader(testDataset, batch_size = batchSize, shuffle = False)

In [5]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

torch.Size([18744, 3, 64, 64])
torch.Size([18744])
torch.Size([2083, 3, 64, 64])
torch.Size([2083])


In [6]:
class CNN(nn.Module):
  def __init__(self, inChannels, numClasses):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels = inChannels, out_channels= 16, kernel_size = 3, stride = 1, padding = 1)
    self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)
    self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3, stride = 1, padding = 1)
    self.fc1 = nn.Linear(32 * 16 * 16, 128)
    self.fc2 = nn.Linear(128, numClasses)
    self.dropfc1 = nn.Dropout(p=0.2)
    # self.dropconv2 = nn.Dropout2d(p=0.1)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = x.view(-1, 32 * 16 * 16)
    x = F.relu(self.dropfc1(self.fc1(x)))
    x = self.fc2(x)
    return x
  def Pri(self):
      print("sususu")

model = CNN(inChannels = 3, numClasses = 10).to(device)
print(model)

CNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=8192, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
  (dropfc1): Dropout(p=0.2, inplace=False)
)


In [7]:
#Hyper parameters
learningRate = 1e-3
epochs = 70
lossFunction = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learningRate)
scheduler = lr_scheduler.LinearLR(optimizer,start_factor = 1.0, end_factor= 0.1, total_iters= 50)
losses = []

In [10]:
def train():
  for epoch in range(epochs):
    epochLoss = 0
    numBatches = 0
    for X, y in trainLoader:
      optimizer.zero_grad()
      pred = model(X)
      loss = lossFunction(pred, y)
      loss.backward()
      optimizer.step()
      epochLoss += loss.item()
      numBatches += 1
    scheduler.step()
    avgLoss = epochLoss/numBatches
    losses.append(avgLoss)
    print(f'Epoch {epoch+1}, Loss: {avgLoss:.9f}, Lr: {optimizer.param_groups[0]["lr"]:.6}')

train()

Epoch 1, Loss: 0.029363819, Lr: 0.000982
Epoch 2, Loss: 0.020720524, Lr: 0.000964
Epoch 3, Loss: 0.031179596, Lr: 0.000946
Epoch 4, Loss: 0.011550658, Lr: 0.000928
Epoch 5, Loss: 0.022334508, Lr: 0.00091
Epoch 6, Loss: 0.012342612, Lr: 0.000892
Epoch 7, Loss: 0.019630678, Lr: 0.000874
Epoch 8, Loss: 0.017632081, Lr: 0.000856
Epoch 9, Loss: 0.009886972, Lr: 0.000838
Epoch 10, Loss: 0.018860901, Lr: 0.00082
Epoch 11, Loss: 0.016754675, Lr: 0.000802
Epoch 12, Loss: 0.018825842, Lr: 0.000784
Epoch 13, Loss: 0.009570366, Lr: 0.000766
Epoch 14, Loss: 0.015139647, Lr: 0.000748
Epoch 15, Loss: 0.014139303, Lr: 0.00073
Epoch 16, Loss: 0.008893348, Lr: 0.000712
Epoch 17, Loss: 0.010071772, Lr: 0.000694
Epoch 18, Loss: 0.014410808, Lr: 0.000676
Epoch 19, Loss: 0.012974727, Lr: 0.000658
Epoch 20, Loss: 0.012572528, Lr: 0.00064
Epoch 21, Loss: 0.006358754, Lr: 0.000622
Epoch 22, Loss: 0.010961068, Lr: 0.000604
Epoch 23, Loss: 0.012290421, Lr: 0.000586
Epoch 24, Loss: 0.006612038, Lr: 0.000568
Epoch

In [11]:
def check_accuracy():


    num_correct = 0
    num_samples = 0
    model.eval()  # Set the model to evaluation mode

    with torch.no_grad():  # Disable gradient calculation
        for x, y in testLoader:

            # Forward pass: compute the model output
            scores = model(x)
            _, predictions = scores.max(1)  # Get the index of the max log-probability
            num_correct += (predictions == y).sum()  # Count correct predictions
            num_samples += predictions.size(0)  # Count total samples

        # Calculate accuracy
        accuracy = float(num_correct) / float(num_samples) * 100 
        print(f"Got {num_correct}/{num_samples} with accuracy {accuracy:.2f}%")

    model.train()  # Set the model back to training mode

# Final accuracy check on training and test sets
check_accuracy()


Got 1831/2083 with accuracy 87.90%
